In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
from datasets import load_dataset

## Loading the dataset
The dataset in question can be found at this link: https://huggingface.co/datasets/VLyb/FB15k  and it contains entities in the form [HEAD], [RELATION], [TAIL]. This means that the two entities head and tail are connected by a relation of a given type. In this version we have a total of 237 types of relations.

In [4]:
dataset = load_dataset("VLyb/FB15k-237")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

valid.csv:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/272115 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/17535 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20466 [00:00<?, ? examples/s]